# XGBoost (Gradient Boosted Decision Trees) 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

train_data = pd.read_csv('train.csv')
submission_data = pd.read_csv('test.csv')

#removing rows where sale price not available
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

y = train_data.SalePrice
X = train_data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

numeric_cols = [cname for cname in submission_data.columns if 
                                submission_data[cname].dtype in ['int64', 'float64']]
submission_X = submission_data[numeric_cols]

#print(type(y))
#print(type(submission_X))

In [2]:
train_X, test_X, train_y, test_y = train_test_split(X, y.as_matrix(), test_size=0.25)

my_imputer = Imputer()
train_X = my_imputer.fit_transform(train_X)
test_X = my_imputer.transform(test_X)
submission_X = my_imputer.transform(submission_X)



#rint(type(X))
#print(type(submission_X))

In [3]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

/home/lazyowl/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [4]:
# make predictions
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

Mean Absolute Error : 16804.9110017


## Tuning XGBoost

In [5]:
#adding n_estimators and early_stopping_rounds
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)])

[0]	validation_0-rmse:180127
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:163178
[2]	validation_0-rmse:148071
[3]	validation_0-rmse:134366
[4]	validation_0-rmse:121776
[5]	validation_0-rmse:110703
[6]	validation_0-rmse:100909
[7]	validation_0-rmse:92121.5
[8]	validation_0-rmse:84530.7
[9]	validation_0-rmse:77713.8
[10]	validation_0-rmse:71691.8
[11]	validation_0-rmse:66388.2
[12]	validation_0-rmse:61786.9
[13]	validation_0-rmse:57560.1
[14]	validation_0-rmse:53787.1
[15]	validation_0-rmse:50706.8
[16]	validation_0-rmse:47694.1
[17]	validation_0-rmse:45319.8
[18]	validation_0-rmse:43072.9
[19]	validation_0-rmse:41381.9
[20]	validation_0-rmse:39911.9
[21]	validation_0-rmse:38732.7
[22]	validation_0-rmse:37664.2
[23]	validation_0-rmse:36774.1
[24]	validation_0-rmse:35932.8
[25]	validation_0-rmse:35426.6
[26]	validation_0-rmse:34881.3
[27]	validation_0-rmse:34474.7
[28]	validation_0-rmse:33931.6
[29]	validation_0-rmse:33562.1
[30]	validation_0-rmse:

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [6]:
# iteration 72 is best 
my_model = XGBRegressor(n_estimators=1000)
# Add silent=True to avoid printing out updates with each cycle
#this is done to prevent leaky validation stratregy
X = my_imputer.fit_transform(X)
my_model.fit(X, y, verbose=False)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [7]:
print(X.shape)
print(submission_X.shape)

(1460, 37)
(1459, 37)


In [8]:
# Use the model to make predictions
predicted_prices = my_model.predict(submission_X)
# We will look at the predicted prices to ensure we have something sensible.
print(predicted_prices)

[ 124559.5625    165771.921875  183148.75     ...,  175554.578125
  107897.734375  231158.59375 ]


# Submission

In [9]:
my_submission = pd.DataFrame({'Id': submission_data.Id, 'SalePrice': predicted_prices})
# you could use any filename. We choose submission here
my_submission.to_csv('submission_XGBoost.csv', index=False)